In [2]:
from google.colab import drive
import os

# 1. Mount and Clone
drive.mount('/content/drive')
!git clone https://github.com/Usaid-Siddiqui/QG_Ijepa.git /content/QG_IJEPA
%cd /content/QG_IJEPA

# 2. Setup Data (Copy subset to local SSD)
!cp /content/drive/MyDrive/QG_IJEPA/data/qg_medium_train.h5 /content/qg_compressed.h5
!cp /content/drive/MyDrive/QG_IJEPA/data/qg_medium_finetune.h5 /content/qg_compressed_finetune.h5
!cp /content/drive/MyDrive/QG_IJEPA/data/qg_medium_test.h5 /content/qg_compressed_test.h5


# 3. Pathing
os.environ['PYTHONPATH'] += ":/content/QG_IJEPA"

Mounted at /content/drive
Cloning into '/content/QG_IJEPA'...
remote: Enumerating objects: 248, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 248 (delta 5), reused 9 (delta 4), pack-reused 232 (from 1)
Receiving objects: 100% (248/248), 49.77 KiB | 7.11 MiB/s, done.
Resolving deltas: 100% (150/150), done.
/content/QG_IJEPA


In [11]:
!git restore /content/QG_IJEPA/colab_config.yaml
!git pull

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0 (from 0)
Unpacking objects: 100% (3/3), 430 bytes | 430.00 KiB/s, done.
From https://github.com/Usaid-Siddiqui/QG_Ijepa
   0da9c0e..92c8561  main       -> origin/main
Updating 0da9c0e..92c8561
Fast-forward
 train_pretrain.py | 7 ++++++-
 1 file changed, 6 insertions(+), 1 deletion(-)


In [9]:
# We need to decompress the data and store the in chunks optimal for the dataloader to read

from utils import unpack_h5
unpack_h5("/content/qg_compressed.h5", "/content/qg_fast.h5")
unpack_h5("/content/qg_compressed_test.h5", "/content/qg_fast_test.h5")
unpack_h5("/content/qg_compressed_finetune.h5", "/content/qg_fast_finetune.h5")

Target file /content/qg_fast.h5 already exists. Skipping.
Unpacking 20000 samples to /content/qg_fast_test.h5...
Progress: 0/20000
Progress: 5000/20000
Progress: 10000/20000
Progress: 15000/20000
Unpacking Complete.
Unpacking 20000 samples to /content/qg_fast_finetune.h5...
Progress: 0/20000
Progress: 5000/20000
Progress: 10000/20000
Progress: 15000/20000
Unpacking Complete.


In [12]:
# config_colab.yaml
%%writefile colab_config.yaml
data:
  h5_path: "/content/qg_fast.h5"
  batch_size: 256
  num_workers: 2

model:
  name: "vit_small_ijepa"
  patch_dim: 192
  embed_dim: 384
  depth: 12
  num_heads: 6
  predictor_embed_dim: 192
  predictor_depth: 6

train:
  base_lr: 0.00005
  epochs: 25
  warmup_epochs: 8
  ema_momentum: 0.999
  weight_decay: 0.001
  save_freq: 5
  # Paths in Drive for persistent storage
  checkpoint_dir: "/content/drive/MyDrive/QG_IJEPA/checkpoints"
  log_dir: "/content/drive/MyDrive/QG_IJEPA/logs"

  # Optional: Resume from a specific checkpoint
  # resume_from: "/content/drive/MyDrive/QG_IJEPA/checkpoints/20260122_0154/checkpoint_epoch_8.pth"

masking:
  grid_size: 16
  num_targets: 2
  target_scale: [0.1, 0.15]
  context_scale: [0.5, 0.6]

finetune:
  checkpoint_to_load: "/content/drive/MyDrive/QG_IJEPA/checkpoints/vit_small_ijepa_20260203_040412/best_encoder.pth"
  train_h5_path: "/content/qg_fast_finetune.h5"       # Just for the Linear Head
  test_h5_path: "/content/qg_fast_test.h5"            # The "Vault" for ROC AUC
  batch_size: 64
  epochs: 5
  lr: 0.001
  freeze_encoder: True

Overwriting colab_config.yaml


In [ ]:
!python -u train_pretrain.py

2026-02-03 06:03:14,081 | INFO | Config saved to /content/drive/MyDrive/QG_IJEPA/checkpoints/vit_small_ijepa_20260203_060314
2026-02-03 06:03:14,088 | INFO | --- Dataset Loaded (Size: 100000) ---
Epoch 1: 100% 391/391 [05:07<00:00,  1.27batch/s]
2026-02-03 06:08:23,785 | INFO | Epoch [1/25] | Loss: 1.081902 | LR: 0.000000
2026-02-03 06:08:27,388 | INFO | --- New best encoder saved at epoch 1 ---
Epoch 2: 100% 391/391 [05:05<00:00,  1.28batch/s]
2026-02-03 06:13:33,356 | INFO | Epoch [2/25] | Loss: 0.527054 | LR: 0.000006
2026-02-03 06:13:36,840 | INFO | --- New best encoder saved at epoch 2 ---
Epoch 3: 100% 391/391 [05:08<00:00,  1.27batch/s]
2026-02-03 06:18:44,995 | INFO | Epoch [3/25] | Loss: 0.301482 | LR: 0.000013
2026-02-03 06:18:49,881 | INFO | --- New best encoder saved at epoch 3 ---
Epoch 4: 100% 391/391 [05:05<00:00,  1.28batch/s]
2026-02-03 06:23:55,759 | INFO | Epoch [4/25] | Loss: 0.200616 | LR: 0.000019
2026-02-03 06:23:59,310 | INFO | --- New best encoder saved at epoc

Time for finetuning and evaluation.

In [10]:
!python train_finetune.py

--- Training MLP Head | Results path: /content/drive/MyDrive/QG_IJEPA/finetune/vit_small_ijepa_20260203_040412 ---
Epoch 1: 100% 313/313 [01:32<00:00,  3.37it/s, acc=0.612, loss=0.644]
Epoch 2: 100% 313/313 [01:34<00:00,  3.32it/s, acc=0.638, loss=0.592]
Epoch 3: 100% 313/313 [01:34<00:00,  3.30it/s, acc=0.648, loss=0.636]
Epoch 4: 100% 313/313 [01:34<00:00,  3.30it/s, acc=0.65, loss=0.557]
--- Computing Final ROC AUC ---
100% 313/313 [01:34<00:00,  3.30it/s]
Figure(800x600)

Final AUC: 0.7074
Results saved to: /content/drive/MyDrive/QG_IJEPA/finetune/vit_small_ijepa_20260203_040412


In [ ]:
import h5py
import numpy as np
from tqdm import tqdm

def calculate_stats(h5_path):
    with h5py.File(h5_path, 'r') as f:
        data = f['images'] # [N, 125, 125, 3]

        # We calculate per channel (Tracks, ECAL, HCAL)
        sums = np.zeros(3)
        sq_sums = np.zeros(3)
        num_pixels = len(data) * 125 * 125

        for i in tqdm(range(len(data))):
            img = data[i].astype(np.float32)
            # Apply log1p first since that's what the model will see
            img = np.log1p(img)

            sums += np.sum(img, axis=(0, 1))
            sq_sums += np.sum(img**2, axis=(0, 1))

        mean = sums / num_pixels
        std = np.sqrt((sq_sums / num_pixels) - mean**2)

        return mean, std

mean, std = calculate_stats("/content/qg_fast.h5")
print(f"Mean: {mean}, Std: {std}")

100%|██████████| 100000/100000 [01:57<00:00, 854.05it/s]

Mean: [0.01122842 0.01998984 0.08350217], Std: [0.17538942 0.21435375 0.55736226]
